# **2D-CNN**

In [2]:
import numpy as np
import mne
import pandas as pd
import mne_bids
import matplotlib.pyplot as plt
import os
import MelFilterBank as mel
import reconstructWave as rW
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, InputLayer, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils.vis_utils import plot_model
import torch
import sys
import soundfile as sf
import skimage.transform
from tensorflow import keras
import gc
import WaveGlow_functions
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import datetime
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers import Dense
import argparse
from subprocess import call, check_output, run
import scipy.stats
import scipy.io.wavfile
import scipy.fftpack
import scipy.io as sio
import scipy
import librosa
import librosa.display
import tensorflow as tf

# Additional imports
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import mne
import numpy as np
import scipy

import soundfile as sf

# Set TensorFlow GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


KeyboardInterrupt: 

In [ ]:
# do not use all GPU memory
from tensorflow.compat.v1.keras.backend import set_session
import tensorflow.compat.v1 as tf
config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config)
set_session(sess)

### **Helper functions**

In [ ]:
#Small helper function to speed up the hilbert transform by extending the length of data to the next power of 2
hilbert3 = lambda x: scipy.signal.hilbert(x, scipy.fftpack.next_fast_len(len(x)),axis=0)[:len(x)]

In [ ]:
def extractHG(data, sr, windowLength=0.05, frameshift=0.01, bandpass_min=70, bandpass_max=170):
    """
    Window data and extract frequency-band envelope using the hilbert transform
    
    Parameters
    ----------
    data: array (samples, channels)
        EEG time series
    sr: int
        Sampling rate of the data
    windowLength: float
        Length of window (in seconds) in which spectrogram will be calculated
    frameshift: float
        Shift (in seconds) after which next window will be extracted
    Returns
    ----------
    feat: array (windows, channels)
        Frequency-band feature matrix
    """
    #Linear detrend
    data = scipy.signal.detrend(data,axis=0)
    #Number of windows
    numWindows = int(np.floor((data.shape[0]-windowLength*sr)/(frameshift*sr)))
    #Filter High-Gamma Band
    # sos = scipy.signal.iirfilter(4, [70/(sr/2),170/(sr/2)],btype='bandpass',output='sos')
    sos = scipy.signal.iirfilter(4, [bandpass_min/(sr/2),bandpass_max/(sr/2)],btype='bandpass',output='sos')
    data = scipy.signal.sosfiltfilt(sos,data,axis=0)
    #Attenuate first harmonic of line noise
    # sos = scipy.signal.iirfilter(4, [98/(sr/2),102/(sr/2)],btype='bandstop',output='sos')
    # data = scipy.signal.sosfiltfilt(sos,data,axis=0)
    #Attenuate second harmonic of line noise
    # sos = scipy.signal.iirfilter(4, [148/(sr/2),152/(sr/2)],btype='bandstop',output='sos')
    # data = scipy.signal.sosfiltfilt(sos,data,axis=0)
    #Create feature space
    data = np.abs(hilbert3(data))
    feat = np.zeros((numWindows,data.shape[1]))
    for win in range(numWindows):
        start= int(np.floor((win*frameshift)*sr))
        stop = int(np.floor(start+windowLength*sr))
        feat[win,:] = np.mean(data[start:stop,:],axis=0)
    return feat

In [ ]:
def stackFeatures(features, modelOrder=4, stepSize=5):
    """
    Add temporal context to each window by stacking neighboring feature vectors
    
    Parameters
    ----------
    features: array (windows, channels)
        Feature time series
    modelOrder: int
        Number of temporal context to include prior to and after current window
    stepSize: float
        Number of temporal context to skip for each next context (to compensate for frameshift)
    Returns
    ----------
    featStacked: array (windows, feat*(2*modelOrder+1))
        Stacked feature matrix
    """
    featStacked=np.zeros((features.shape[0]-(2*modelOrder*stepSize),(2*modelOrder+1)*features.shape[1]))
    for fNum,i in enumerate(range(modelOrder*stepSize,features.shape[0]-modelOrder*stepSize)):
        ef=features[i-modelOrder*stepSize:i+modelOrder*stepSize+1:stepSize,:]
        featStacked[fNum,:]=ef.flatten() #Add 'F' if stacked the same as matlab
    return featStacked

In [ ]:
# WaveGlow / Tacotron2 / STFT parameters for audio data
# samplingFrequency = 16000
samplingFrequency = 22050
#samplingFrequency_EEG = 512 #sub 07
winL_EEG = 0.05
# frameshift_EEG = 0.01 # 10 ms
frameshift_EEG = 0.01 # 10 ms
frameshift_speech = 220 # 10ms
# modelOrder_EEG = 1
# modelOrder_EEG = 2
modelOrder_EEG = 4
# modelOrder_EEG = 10
stepSize_EEG = 5

In [ ]:
stft = WaveGlow_functions.TacotronSTFT(
        filter_length=1024,
        hop_length=frameshift_speech,
        win_length=1024,
        n_mel_channels=80,
        sampling_rate=samplingFrequency,
        mel_fmin=0,
        mel_fmax=8000)

### **Load Data**

In [ ]:
from pydub import AudioSegment

# Load Audio
wavfile = 'data/stimuli/6min.wav'
audio = AudioSegment.from_file(wavfile, format='wav')

# Shift audio by 150 ms
shifted_audio = audio._spawn(audio.raw_data, overrides={'frame_rate': audio.frame_rate, 'frame_width': audio.sample_width})
shifted_audio = shifted_audio._spawn(shifted_audio.raw_data, overrides={'frame_rate': shifted_audio.frame_rate + int(22050*0.15)})
shifted_audio = shifted_audio.set_frame_rate(audio.frame_rate)
shifted_audio = shifted_audio.set_channels(audio.channels)

# Export shifted audio as WAV file
shifted_audio.export('shifted_audio.wav', format='wav')

In [ ]:
#Load Audio
wavfile = 'shifted_audio.wav'
mel_data = WaveGlow_functions.get_mel(wavfile, stft)
mel_data = np.fliplr(np.rot90(mel_data.data.numpy(), axes=(1, 0)))


# Print out the duration of the original and shifted audio
print("Original audio duration:", audio.duration_seconds)
print("Shifted audio duration:", shifted_audio.duration_seconds)

In [ ]:
plt.imshow(mel_data, aspect='auto')
plt.xlabel('Frame')
plt.ylabel('Mel Bin')
plt.title('Mel Spectrogram of the Audio')
plt.show()

In [ ]:
plt.imshow(np.transpose(mel_data), aspect='auto')
plt.xlabel('Mel Bin')
plt.ylabel('Frame')
plt.title('Transposed Mel Spectrogram of the Audio')
plt.show()


In [ ]:
#Load Subjects
bids_dir = 'data'
subjects = mne_bids.get_entity_vals(bids_dir, 'subject')
print(subjects)

In [ ]:
#Choose subjects
subject = '55'
acquisition = 'clinical'
task = 'film'
datatype = 'ieeg'
session = 'iemu'

In [ ]:
#load subject's channels
channels_path = mne_bids.BIDSPath(subject=subject,
                                    session=session,
                                    suffix='channels',
                                    extension='tsv',
                                    datatype=datatype,
                                    task=task,
                                    acquisition=acquisition,
                                    root=bids_dir)

In [ ]:
channels = pd.read_csv(str(channels_path.match()[0]), sep='\t', header=0, index_col=None)
#print(channels)

In [ ]:
#set channel types
data_path = mne_bids.BIDSPath(subject=subject,
                                    session=session,
                                    suffix='ieeg',
                                    extension='vhdr',
                                    datatype=datatype,
                                    task=task,
                                    acquisition=acquisition,
                                    root=bids_dir)
raw = mne.io.read_raw_brainvision(str(data_path.match()[0]), scale=1.0, preload=False, verbose=True)
raw.set_channel_types({ch_name: str(x).lower()
                if str(x).lower() in ['ecog', 'seeg', 'eeg'] else 'misc'
                                for ch_name, x in zip(raw.ch_names, channels['type'].values)})
raw.drop_channels([raw.ch_names[i] for i, j in enumerate(raw.get_channel_types()) if j == 'misc'])

### Discard Bad Channels

In [ ]:
#bad channels
bad_channels = channels['name'][(channels['type'].isin(['ECOG', 'SEEG'])) & (channels['status'] == 'bad')].tolist()
raw.info['bads'].extend([ch for ch in bad_channels])
raw.drop_channels(raw.info['bads'])

### Load Raw Data

In [ ]:
raw.load_data()

In [ ]:
n_eeg_channels = int(raw.info['nchan']) # for subject 01
print('n_eeg_channels', n_eeg_channels)
# raise

### Apply notch filter to remove line noise

In [ ]:
raw.notch_filter(freqs=np.arange(50, 251, 50))

raw.plot()
plt.show()

### Apply common average reference to remove common noise and trends

In [ ]:
#CAR
raw_car, _ = mne.set_eeg_reference(raw.copy(), 'average')

In [ ]:
gamma = raw_car.copy().filter(60, 120).apply_hilbert(envelope=True).get_data()#.T
print('raw_car.shape:', raw_car._data.shape, 'gamma shape: ', gamma.shape)

#Extract signal in gamma range, use Hilbert transform, but can also play around with wavelet decomposition options


gamma = raw_car.copy().filter(60, 120).apply_hilbert(envelope=True).get_data().T

### Read annotation with event markers

In [ ]:
custom_mapping = {'Stimulus/music': 2, 'Stimulus/speech': 1,
                  'Stimulus/end task': 5}  # 'Stimulus/task end' in laan
events, event_id = mne.events_from_annotations(raw_car, event_id=custom_mapping,
                                                         use_rounding=False)

In [ ]:
#plotting
raw_car.plot(n_channels=30,scalings='auto', duration=3, start=29)
plt.show()

In [ ]:
# assume gamma is a 2D array
np.savetxt("gamma.tsv", gamma, delimiter="\t")

n_melspec = 80
#get EEG SR
samplingFrequency_EEG=raw_car.info['sfreq']

### Crop to keep only the segments while wathcing the stimuli ( 6.5 min long movie)

In [ ]:
#create a copy taht we crop
raw_car_cut = raw_car._data.copy()
print(raw_car_cut.shape)

In [ ]:
print('before cut: ', raw_car._data.shape, mel_data.shape)

raw_car_cut = np.empty((n_eeg_channels,0))
mel_data_cut = np.empty((0,n_melspec))


# for i in range(6):
for i in range(6):
    start_time = events[2*i+1, 0] / raw_car.info['sfreq']
    end_time = events[2*i+2, 0] / raw_car.info['sfreq']
    start_idx, end_idx = raw_car.time_as_index([start_time, end_time])
    print(i, 'iEEG index', start_idx, end_idx, end_idx-start_idx)
    n_frames_per_sec = int(1 / frameshift_EEG)
    print(i, 'melspec index', (2*i+1)*30*n_frames_per_sec, (2*i+2)*30*n_frames_per_sec, (2*i+2)*30*n_frames_per_sec-(2*i+1)*30*n_frames_per_sec)
    # raw_car_cut1 = raw_car._data[:, start_idx:end_idx]
    raw_car_cut1 = gamma[:, start_idx:end_idx]
    raw_car_cut = np.append(raw_car_cut, raw_car_cut1, axis=1)
    mel_data_cut1 = mel_data[(2*i+1)*30*n_frames_per_sec : (2*i+2)*30*n_frames_per_sec]
    mel_data_cut = np.append(mel_data_cut, mel_data_cut1, axis=0)
# raise
mel_data = mel_data_cut

print('after cut: ', raw_car_cut.shape, mel_data.shape)
# raise
#praat

#get EEG SR
samplingFrequency_EEG=raw_car.info['sfreq']

# Calculate the length of the signal
length = raw_car_cut.shape[1] / samplingFrequency_EEG 
print("The length of the EEG signal is", length,"s")
print(samplingFrequency_EEG)

### Extract features

In [ ]:
#Extract HG features
print('calculating Hilbert...', raw_car_cut.shape)
# eeg_fft = np.empty((n_max_frames, n_freq_bands, n_eeg_channels * (2 * modelOrder_EEG + 1) ))
# feat_Hilbert_1 = extractHG(raw_car_cut,samplingFrequency_EEG, windowLength=winL_EEG,frameshift=frameshift_EEG, bandpass_min=1, bandpass_max=200)
feat_Hilbert_1 = extractHG(np.rot90(raw_car_cut),samplingFrequency_EEG, windowLength=winL_EEG,frameshift=frameshift_EEG, bandpass_min=1, bandpass_max=200)
# feat_Hilbert_2 = extractHG(np.rot90(current_raw_eeg_data),samplingFrequency_EEG, windowLength=winL_EEG,frameshift=frameshift_EEG, bandpass_min=51, bandpass_max=100)
# feat_Hilbert_3 = extractHG(np.rot90(current_raw_eeg_data),samplingFrequency_EEG, windowLength=winL_EEG,frameshift=frameshift_EEG, bandpass_min=101, bandpass_max=150)
# feat_Hilbert_4 = extractHG(np.rot90(current_raw_eeg_data),samplingFrequency_EEG, windowLength=winL_EEG,frameshift=frameshift_EEG, bandpass_min=151, bandpass_max=200)

In [ ]:
#Stack features
feat_Hilbert_1 = stackFeatures(feat_Hilbert_1,modelOrder=modelOrder_EEG,stepSize=stepSize_EEG)

In [ ]:

plt.subplot(211)
plt.imshow(np.rot90(feat_Hilbert_1), aspect='auto')

plt.subplot(212)
plt.imshow(np.rot90(mel_data).T, aspect='auto')
plt.show()


eeg = feat_Hilbert_1

In [ ]:
min_len = np.min((len(eeg), len(mel_data)))
eeg = eeg[0:min_len]
mel_data = mel_data[0:min_len]

In [ ]:
print('mel & iEEG: ', mel_data.shape, feat_Hilbert_1.shape)

In [ ]:
train_index = np.arange(0, int(0.8 * eeg.shape[0]))
test_index = np.arange(int(0.8 * eeg.shape[0]), eeg.shape[0])

In [ ]:
# train-validation-test split
eeg_train = eeg[0 : int(len(eeg) * 0.8)]
eeg_valid = eeg[int(len(eeg) * 0.8) : int(len(eeg) * 0.9)]
eeg_test =  eeg[int(len(eeg) * 0.9) : ]    

In [ ]:
melspec_train = mel_data[0 : int(len(mel_data) * 0.8)]
melspec_valid = mel_data[int(len(mel_data) * 0.8) : int(len(mel_data) * 0.9)]
melspec_test =  mel_data[int(len(mel_data) * 0.9) : ]    

In [ ]:
# scale input to [0-1]
eeg_scaler = MinMaxScaler()
# eeg_scaler = StandardScaler(with_mean=True, with_std=True)
eeg_train_scaled = eeg_scaler.fit_transform(eeg_train)
eeg_valid_scaled = eeg_scaler.transform(eeg_valid)
eeg_test_scaled  = eeg_scaler.transform(eeg_test)

In [ ]:
# scale outpit mel-spectrogram data to zero mean, unit variances
melspec_scaler = StandardScaler(with_mean=True, with_std=True)
melspec_train_scaled = melspec_scaler.fit_transform(melspec_train)
melspec_valid_scaled = melspec_scaler.transform(melspec_valid)
melspec_test_scaled  = melspec_scaler.transform(melspec_test)

# **2D CNN**

In [ ]:
import os
import numpy as np
import MelFilterBank as mel
import reconstructWave as rW
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, InputLayer, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import datetime
from keras.utils.vis_utils import plot_model
import torch
import sys
import soundfile as sf
import skimage.transform
from tensorflow import keras
import gc

In [ ]:
def strided_app(a, L, S, verbose=None):  # Window len = L, Stride len/stepsize = S
    shape = a.shape[1:]
    nrows = ((a.shape[0] - L) // S) + 1
    strides = a.strides
    #print(shape, strides)
    if verbose:
        print("strides:", strides)
    return np.lib.stride_tricks.as_strided(a, shape=(nrows, L) + shape,
                                           strides=(S * strides[0],) + strides)

In [ ]:
method = '2D-CNN'
result_path = os.path.join(os.getcwd(), f"results_{method}")
winLength = 0.05
frameshift = 0.01
audiosr = 16000

spectrogram = mel_data
data = eeg
pt=subject

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [ ]:
    # Create a train and test split from data, test is 20% of the data
    train_index = np.arange(0, int(0.8 * data.shape[0]))
    test_index = np.arange(int(0.8 * data.shape[0]), data.shape[0])

    # Initialize an empty spectrogram to save the reconstruction to
    rec_spec = np.zeros(spectrogram.shape)

    # Z-Normalize with mean and std from the training data
    mu = np.mean(data[train_index, :], axis=0)
    std = np.std(data[train_index, :], axis=0)
    trainData = (data[train_index, :] - mu) / std
    testData = (data[test_index, :] - mu) / std

    # Z-Normalize with mean and std from the training data -- output
    mu = np.mean(spectrogram[train_index, :], axis=0)
    std = np.std(spectrogram[train_index, :], axis=0)
    trainSpectrogram = (spectrogram[train_index, :] - mu) / std
    testSpectrogram = (spectrogram[test_index, :] - mu) / std

    print('Input shape: ', trainData.shape)
    print('Input shape: ', testData.shape)
        
    # Find the right shape for the input, as it should be 3D, like 1143 is 9*127
    new_shape = int(trainData.shape[1] / 9)

    # reshape input from 1143 to 9*127
    trainData = trainData.reshape(-1, 9, new_shape)
    testData = testData.reshape(-1, 9, new_shape)
    print('Input shape: ', trainData.shape)

    sts = 6
    window_size = sts * 4 + 1
    n_to_skip = np.floor(window_size // 2).astype(np.int64)

    print('Input shape: ', trainData.shape)

    #conversion to 3D blocks
    trainData = strided_app(trainData, window_size, 1)
    trainSpectrogram = trainSpectrogram[n_to_skip:(trainSpectrogram.shape[0] - n_to_skip)]

    testData = strided_app(testData, window_size, 1)
    testSpectrogram = testSpectrogram[n_to_skip:(testSpectrogram.shape[0] - n_to_skip)]

    print('Input shape: ', trainData.shape)
    print('Input/validation shape: ', testData.shape)
    print('Output shape: ', trainSpectrogram.shape)

In [ ]:
    model = Sequential()
    model.add(InputLayer(input_shape=trainData.shape[1:]))
    model.add(Conv2D(filters=40,
                     kernel_size=(13, 13),
                     strides=(sts, 2),
                     activation=tensorflow.nn.swish,
                     padding="same",
                     kernel_initializer=keras.initializers.he_uniform(seed=None),
                     kernel_regularizer=regularizers.l1(0.00001),
                     input_shape=trainData.shape[1:]))

    model.add(Dropout(0.1))
    model.add(Conv2D(filters=400, kernel_size=(13, 13), strides=(2, 2), activation=tensorflow.nn.swish,
                     padding="same", kernel_initializer=keras.initializers.he_uniform(seed=None),
                     kernel_regularizer=regularizers.l1(0.00001)))
    model.add(Dropout(0.1))
   # model.add(MaxPooling2D(pool_size=(2, 2)))
   # model.add(Conv2D(filters=200, kernel_size=(13, 13), strides=(2, 2), activation=tensorflow.nn.swish,
             #        padding="same", kernel_initializer=keras.initializers.he_uniform(seed=None),
             #        kernel_regularizer=regularizers.l1(0.00001)))
  #  model.add(Dropout(0.1))
    #model.add(Conv3D(filters=100, kernel_size=(1, 13, 13), strides=(1, 1, 1), activation=tensorflow.nn.swish,
                     # padding="same", kernel_initializer=keras.initializers.he_uniform(seed=None),
                     # kernel_regularizer=regularizers.l1(0.00001)))
    #model.add(Dropout(0.2))
    #model.add(MaxPooling3D(pool_size=(1, 2, 2)))
    model.add(Flatten())
    model.add(
        Dense(100000, activation=tensorflow.nn.swish, kernel_initializer=keras.initializers.he_uniform(seed=None),
              bias_initializer=keras.initializers.he_uniform(seed=None),
              kernel_regularizer=regularizers.l1(0.000005)))
    model.add(Dropout(0.1))
    model.add(Dense(trainSpectrogram.shape[1], activation='linear'))

    plot_model(model, to_file=f"model_{method}.png", show_shapes=True, show_layer_names=True)

    model.compile(
            loss='mean_squared_error',
            metrics=['mean_squared_error'],
            optimizer='adam')
    earlystopper = EarlyStopping(monitor='val_mean_squared_error', min_delta=0.001, patience=3, verbose=1,
                                 mode='auto')
    lrr = ReduceLROnPlateau(monitor='val_mean_squared_error', patience=2, verbose=1, factor=0.5, min_lr=0.0001)

    print(model.summary())

    if not (os.path.isdir('models/')):
        os.mkdir('models/')

    # early stopping to avoid over-training
    model_name = 'models/iEEG_to_melspec_2D-CNN_sp-' + pt

    # csapot: temporarily disabled
    checkp = ModelCheckpoint(
        model_name +
        '_weights_best.h5',
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min')

    # Run training
    history = model.fit(trainData, trainSpectrogram,
                        epochs=100, batch_size=64, shuffle=False, verbose=1,
                        callbacks=[earlystopper, checkp, lrr],
                        validation_data=(testData, testSpectrogram),
                        )


    # load back best weights
    model.load_weights(model_name + '_weights_best.h5')

    rec_spec = model.predict(testData)

    # inverse transform
    # testSpectrogram=(spectrogram[test,:]-mu)/std
    rec_spec = rec_spec * std + mu

    print('start saving wav')

    # Save reconstructed spectrogram
    os.makedirs(os.path.join(result_path), exist_ok=True)
    np.save(os.path.join(result_path, f'{pt}_predicted_spec.npy'), rec_spec)


    # remove model file
    os.remove(model_name + '_weights_best.h5')
    del model
    # Run garbage collection
    gc.collect()
    
    

In [ ]:
best_val_mse = min(history.history['val_mean_squared_error'])

fig, axs = plt.subplots(2, 1, figsize=(10, 5))

axs[0].imshow(np.rot90(melspec_test_scaled[0:1000]), aspect='auto')
axs[0].set_title('EEG Test Scaled')

axs[1].imshow(np.rot90(rec_spec[0:1000]), aspect='auto')
axs[1].set_title('2D-CNN')

plt.subplots_adjust(hspace=0.4)
plt.suptitle('2D-CNN results for patient ' + subject + ' ' + f'Best validation MSE: {best_val_mse:.4f}')
plt.savefig(model_name + '_EEG_scaled_plots.png')
plt.show()

### Audio synth

In [ ]:

# Provide the actual mel spectrogram (melspec_predicted) and sampling frequency (samplingFrequency) from your FC-DNN output
samplingFrequency = 512

# Convert mel spectrogram to audio using Griffin-Lim algorithm
audio = librosa.feature.inverse.mel_to_audio(melspec_predicted, sr=samplingFrequency)

# Save the audio to a WAV file
output_file = 'predicted_audio_2dcnn.wav'
sf.write(output_file, audio, samplingFrequency)